# Sistema de Medición Interactiva con Calibración
## Visión por Computador 3009228
### Universidad Nacional de Colombia - Facultad de Minas

**Objetivos:**
1. Implementar un sistema de medición interactiva en imágenes
2. Calibrar el sistema usando objetos de referencia conocidos
3. Validar la precisión del sistema mediante comparación con medidas reales
4. Calcular incertidumbre y error relativo de las mediciones
5. Medir múltiples objetos desconocidos y visualizar resultados


## 1. Instalación de Dependencias

Primero instalamos las bibliotecas necesarias para el análisis de imágenes y mediciones.

In [1]:
# Instalación de dependencias
!pip install -r ../requirements.txt

## 2. Importación de Bibliotecas

Importamos las bibliotecas necesarias para procesamiento de imágenes y visualización.

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
import sys
notebook_path = Path().resolve()
parent_dir = notebook_path.parent
sys.path.insert(0, str(parent_dir))
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✓ Bibliotecas importadas correctamente")
print(f"  - OpenCV: {cv2.__version__}")
print(f"  - NumPy: {np.__version__}")

✓ Bibliotecas importadas correctamente
  - OpenCV: 4.12.0
  - NumPy: 2.2.6


## 3. Definición de Funciones

### 3.1 Función de Medición Interactiva

Esta función permite al usuario seleccionar dos puntos en una imagen y calcula la distancia en píxeles entre ellos.

In [3]:
def medir_distancia_interactiva(img, titulo="Selecciona dos puntos"):
    """
    Permite al usuario seleccionar dos puntos en una imagen y calcula la distancia entre ellos.
    
    Parámetros:
    -----------
    img : numpy.ndarray
        Imagen en formato BGR (OpenCV)
    titulo : str
        Título de la ventana de selección
        
    Retorna:
    --------
    tuple: (distancia_pixels, punto1, punto2)
        - distancia_pixels: float con la distancia euclidiana
        - punto1, punto2: tuplas (x, y) con las coordenadas
    """
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(titulo, fontsize=14, fontweight='bold')
    plt.axis('on')
    plt.grid(alpha=0.3)
    
    print(f"\n{'='*60}")
    print(f"  {titulo}")
    print(f"{'='*60}")
    print("  Instrucciones:")
    print("  - Click izquierdo: seleccionar punto")
    print("  - Click derecho: cancelar último punto")
    print(f"{'='*60}\n")
    
    puntos = plt.ginput(2, timeout=0)
    plt.close()

    if len(puntos) != 2:
        print("⚠️ Debes seleccionar exactamente dos puntos.")
        return None, None, None
    
    (x1, y1), (x2, y2) = puntos
    dist_px = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    print(f"✓ Puntos seleccionados correctamente:")
    print(f"  Punto 1: ({x1:.2f}, {y1:.2f}) px")
    print(f"  Punto 2: ({x2:.2f}, {y2:.2f}) px")
    print(f"  Distancia: {dist_px:.2f} píxeles\n")
    
    return dist_px, (x1, y1), (x2, y2)

print("✓ Función medir_distancia_interactiva definida")

✓ Función medir_distancia_interactiva definida


### 3.2 Función de Visualización de Resultados

Función para crear visualizaciones completas con todas las mediciones realizadas.

In [4]:
def visualizar_mediciones(img, p1_ref, p2_ref, longitud_ref, 
                         p1_val, p2_val, longitud_val, error_val,
                         objetos_medidos, escala):
    """
    Visualiza todas las mediciones realizadas en la imagen.
    
    Parámetros:
    -----------
    img : numpy.ndarray
        Imagen original
    p1_ref, p2_ref : tuple
        Puntos del objeto de referencia
    longitud_ref : float
        Longitud real del objeto de referencia en cm
    p1_val, p2_val : tuple
        Puntos del objeto de validación
    longitud_val : float
        Longitud medida del objeto de validación en cm
    error_val : float
        Error relativo de validación en porcentaje
    objetos_medidos : list
        Lista de diccionarios con objetos medidos
    escala : float
        Factor de escala cm/pixel
    """
    img_vis = img.copy()
    
    # Dibujar objeto de referencia (azul)
    cv2.line(img_vis, 
             (int(p1_ref[0]), int(p1_ref[1])), 
             (int(p2_ref[0]), int(p2_ref[1])), 
             (255, 0, 0), 3)
    cv2.circle(img_vis, (int(p1_ref[0]), int(p1_ref[1])), 5, (255, 0, 0), -1)
    cv2.circle(img_vis, (int(p2_ref[0]), int(p2_ref[1])), 5, (255, 0, 0), -1)
    cv2.putText(img_vis, f"Ref: {longitud_ref:.1f}cm", 
                (int(p1_ref[0]), int(p1_ref[1])-15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    
    # Dibujar objeto de validación (rojo)
    cv2.line(img_vis, 
             (int(p1_val[0]), int(p1_val[1])), 
             (int(p2_val[0]), int(p2_val[1])), 
             (0, 0, 255), 3)
    cv2.circle(img_vis, (int(p1_val[0]), int(p1_val[1])), 5, (0, 0, 255), -1)
    cv2.circle(img_vis, (int(p2_val[0]), int(p2_val[1])), 5, (0, 0, 255), -1)
    cv2.putText(img_vis, f"Val: {longitud_val:.1f}cm (±{error_val:.1f}%)",
                (int(p1_val[0]), int(p1_val[1])-15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # Dibujar objetos medidos (verde)
    for i, obj in enumerate(objetos_medidos):
        cv2.line(img_vis, 
                 (int(obj["p1"][0]), int(obj["p1"][1])), 
                 (int(obj["p2"][0]), int(obj["p2"][1])), 
                 (0, 255, 0), 3)
        cv2.circle(img_vis, (int(obj["p1"][0]), int(obj["p1"][1])), 5, (0, 255, 0), -1)
        cv2.circle(img_vis, (int(obj["p2"][0]), int(obj["p2"][1])), 5, (0, 255, 0), -1)
        cv2.putText(img_vis, f"Obj{i+1}: {obj['medida_real']:.1f}cm",
                    (int(obj["p1"][0]), int(obj["p1"][1])-15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Visualizar
    plt.figure(figsize=(15, 10))
    plt.imshow(cv2.cvtColor(img_vis, cv2.COLOR_BGR2RGB))
    plt.title('Sistema de Medición: Calibración, Validación y Mediciones', 
              fontsize=16, fontweight='bold')
    plt.axis('off')
    
    # Agregar leyenda
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='blue', label=f'Referencia (calibración)'),
        Patch(facecolor='red', label=f'Validación (error: ±{error_val:.2f}%)'),
        Patch(facecolor='green', label='Objetos medidos')
    ]
    plt.legend(handles=legend_elements, loc='upper right', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ Visualización completada")

print("✓ Función visualizar_mediciones definida")

✓ Función visualizar_mediciones definida


## 4. Carga de la Imagen

Cargamos la imagen sobre la cual se realizarán las mediciones. Asegúrate de que la ruta sea correcta.

In [ ]:

# Cargar imagen
image_path = f"{parent_dir}/data/original/IMG01.jpg"
print(f"Cargando imagen desde: {image_path}")
img_bgr = cv2.imread(image_path)

if img_bgr is None:
    raise FileNotFoundError(f"❌ No se pudo cargar la imagen desde: {image_path}")

# Información de la imagen
height, width = img_bgr.shape[:2]
print(f"\n✓ Imagen cargada exitosamente")
print(f"  Dimensiones: {width} x {height} píxeles")
print(f"  Canales: {img_bgr.shape[2]}")
print(f"  Tipo de datos: {img_bgr.dtype}")

# Visualizar imagen original
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
plt.title('Imagen Original para Medición', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

Cargando imagen desde: /Users/carlosviera/Documents/GitHub/registro-imagenes/data/IMG01.jpg


[ WARN:0@0.203] global loadsave.cpp:275 findDecoder imread_('/Users/carlosviera/Documents/GitHub/registro-imagenes/data/IMG01.jpg'): can't open/read file: check file path/integrity


FileNotFoundError: ❌ No se pudo cargar la imagen desde: /Users/carlosviera/Documents/GitHub/registro-imagenes/data/IMG01.jpg

## 5. Calibración del Sistema

### 5.1 Selección del Objeto de Referencia

Para calibrar el sistema, necesitamos un objeto de dimensiones conocidas. Selecciona los extremos de este objeto en la imagen.

In [ ]:
print("\n" + "="*70)
print("  CALIBRACIÓN DEL SISTEMA")
print("="*70)
print("\nSelecciona los extremos del objeto de referencia.")
print("Ejemplo: extremos de una regla, cuadro, o cualquier objeto de dimensión conocida.")
print()

# Medir objeto de referencia
dist_ref_px, p1_ref, p2_ref = medir_distancia_interactiva(
    img_bgr, 
    titulo="📏 CALIBRACIÓN: Selecciona los extremos del objeto de referencia"
)

if dist_ref_px is None:
    raise ValueError("❌ No se seleccionó correctamente el objeto de referencia.")

### 5.2 Ingreso de Dimensión Real

Ingresa la longitud real del objeto de referencia en centímetros.

In [ ]:
# Solicitar longitud real
longitud_real_ref = float(input("Introduce la longitud real del objeto de referencia (en cm): "))

# Calcular escala
escala = longitud_real_ref / dist_ref_px

print(f"\n" + "="*70)
print("  ESCALA CALCULADA")
print("="*70)
print(f"  Distancia en píxeles: {dist_ref_px:.2f} px")
print(f"  Longitud real: {longitud_real_ref:.2f} cm")
print(f"  Escala: {escala:.6f} cm/píxel")
print(f"  Escala inversa: {1/escala:.2f} píxeles/cm")
print("="*70 + "\n")

## 6. Validación del Sistema

### 6.1 Selección del Objeto de Validación

Para estimar la incertidumbre del sistema, mediremos otro objeto de dimensiones conocidas.

In [ ]:
print("\n" + "="*70)
print("  VALIDACIÓN DEL SISTEMA")
print("="*70)
print("\nSelecciona los extremos de OTRO objeto de dimensión conocida.")
print("Esto nos permitirá calcular el error y la incertidumbre del sistema.")
print()

# Medir objeto de validación
dist_valid_px, p1_val, p2_val = medir_distancia_interactiva(
    img_bgr,
    titulo="✓ VALIDACIÓN: Selecciona los extremos del objeto de validación"
)

if dist_valid_px is None:
    raise ValueError("❌ No se seleccionó correctamente el objeto de validación.")

### 6.2 Cálculo de Incertidumbre

Comparamos la medición con la dimensión real conocida para calcular el error del sistema.

In [ ]:
# Solicitar longitud real del objeto de validación
longitud_real_valid = float(input("Introduce la longitud real del objeto de validación (en cm): "))

# Calcular longitud medida y error
longitud_medida_valid = dist_valid_px * escala
error_absoluto = abs(longitud_medida_valid - longitud_real_valid)
error_relativo = (error_absoluto / longitud_real_valid) * 100

print(f"\n" + "="*70)
print("  RESULTADOS DE VALIDACIÓN")
print("="*70)
print(f"  Distancia medida en píxeles: {dist_valid_px:.2f} px")
print(f"  Longitud medida (con escala): {longitud_medida_valid:.2f} cm")
print(f"  Longitud real (conocida): {longitud_real_valid:.2f} cm")
print(f"  Error absoluto: {error_absoluto:.2f} cm")
print(f"  Error relativo: ±{error_relativo:.2f}%")
print("="*70 + "\n")

# Interpretación del error
if error_relativo < 2:
    print("✓ Excelente precisión (< 2%)")
elif error_relativo < 5:
    print("✓ Buena precisión (2-5%)")
elif error_relativo < 10:
    print("⚠️ Precisión aceptable (5-10%)")
else:
    print("⚠️ Precisión baja (> 10%) - considerar recalibrar")

## 7. Medición de Objetos Desconocidos

Ahora que el sistema está calibrado y validado, podemos medir objetos de dimensiones desconocidas.

In [ ]:
print("\n" + "="*70)
print("  MEDICIÓN DE OBJETOS DESCONOCIDOS")
print("="*70)
print()

# Solicitar número de objetos a medir
num_obj = int(input("¿Cuántos objetos deseas medir? "))

# Lista para almacenar objetos medidos
objetos_medidos = []

# Medir cada objeto
for i in range(num_obj):
    print(f"\n{'─'*70}")
    print(f"  OBJETO {i+1} de {num_obj}")
    print(f"{'─'*70}\n")
    
    dist_px, p1, p2 = medir_distancia_interactiva(
        img_bgr,
        titulo=f"📐 MEDICIÓN: Objeto {i+1} - Selecciona los extremos"
    )
    
    if dist_px is None:
        print(f"⚠️ Objeto {i+1} no medido. Continuando...\n")
        continue
    
    # Calcular medida real
    medida_real = dist_px * escala
    
    # Calcular incertidumbre
    incertidumbre = medida_real * (error_relativo / 100)
    
    print(f"✓ Medición completada:")
    print(f"  Longitud estimada: {medida_real:.2f} ± {incertidumbre:.2f} cm")
    print(f"  Incertidumbre relativa: ±{error_relativo:.2f}%\n")
    
    # Guardar objeto
    objetos_medidos.append({
        "nombre": f"Objeto {i+1}",
        "dist_px": dist_px,
        "medida_real": medida_real,
        "incertidumbre": incertidumbre,
        "p1": p1,
        "p2": p2
    })

print(f"\n{'='*70}")
print(f"  MEDICIONES COMPLETADAS: {len(objetos_medidos)} objetos")
print(f"{'='*70}\n")

## 8. Visualización de Resultados

Visualizamos todas las mediciones realizadas en la imagen original.

In [ ]:
print("\nGenerando visualización completa...\n")

# Crear visualización
visualizar_mediciones(
    img_bgr,
    p1_ref, p2_ref, longitud_real_ref,
    p1_val, p2_val, longitud_medida_valid, error_relativo,
    objetos_medidos,
    escala
)

## 9. Resumen y Reporte Final

Generamos un reporte completo con todos los resultados obtenidos.

In [ ]:
print("\n" + "="*70)
print("  REPORTE FINAL - SISTEMA DE MEDICIÓN INTERACTIVA")
print("="*70)

print("\n1. CALIBRACIÓN:")
print(f"   Escala del sistema: {escala:.6f} cm/píxel")
print(f"   Objeto de referencia: {longitud_real_ref:.2f} cm")

print("\n2. VALIDACIÓN:")
print(f"   Longitud real: {longitud_real_valid:.2f} cm")
print(f"   Longitud medida: {longitud_medida_valid:.2f} cm")
print(f"   Error absoluto: {error_absoluto:.2f} cm")
print(f"   Incertidumbre del sistema: ±{error_relativo:.2f}%")

print("\n3. MEDICIONES REALIZADAS:")
if len(objetos_medidos) > 0:
    print(f"   Total de objetos medidos: {len(objetos_medidos)}")
    print()
    for obj in objetos_medidos:
        print(f"   • {obj['nombre']}:")
        print(f"     - Medida: {obj['medida_real']:.2f} ± {obj['incertidumbre']:.2f} cm")
        print(f"     - Distancia en píxeles: {obj['dist_px']:.2f} px")
else:
    print("   No se midieron objetos adicionales")

print("\n4. ESPECIFICACIONES TÉCNICAS:")
print(f"   Resolución de imagen: {width} x {height} píxeles")
print(f"   Resolución espacial: {1/escala:.2f} píxeles/cm")
print(f"   Precisión mínima detectable: {escala:.4f} cm/píxel")

print("\n5. OBSERVACIONES:")
print("   • La incertidumbre estimada se basa en la validación con un objeto")
print("   • Para mayor precisión, considerar promediar múltiples mediciones")
print("   • La precisión depende de la calidad de la imagen y la perspectiva")
print("   • Se recomienda usar objetos de referencia en el mismo plano")

print("\n" + "="*70)
print("  FIN DEL REPORTE")
print("="*70 + "\n")

## 10. Exportar Resultados (Opcional)

Guardar la imagen con las mediciones y un archivo CSV con los resultados.

In [ ]:
import pandas as pd
from datetime import datetime

# Crear directorio de salida
output_dir = Path("resultados_medicion")
output_dir.mkdir(exist_ok=True)

# Timestamp para archivos
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 1. Guardar imagen con mediciones
img_vis = img_bgr.copy()

# Dibujar todas las mediciones
cv2.line(img_vis, (int(p1_ref[0]), int(p1_ref[1])), 
         (int(p2_ref[0]), int(p2_ref[1])), (255, 0, 0), 3)
cv2.putText(img_vis, f"Ref: {longitud_real_ref:.1f}cm", 
            (int(p1_ref[0]), int(p1_ref[1])-15),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

cv2.line(img_vis, (int(p1_val[0]), int(p1_val[1])), 
         (int(p2_val[0]), int(p2_val[1])), (0, 0, 255), 3)
cv2.putText(img_vis, f"Val: {longitud_medida_valid:.1f}cm",
            (int(p1_val[0]), int(p1_val[1])-15),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

for obj in objetos_medidos:
    cv2.line(img_vis, (int(obj["p1"][0]), int(obj["p1"][1])), 
             (int(obj["p2"][0]), int(obj["p2"][1])), (0, 255, 0), 3)
    cv2.putText(img_vis, f"{obj['medida_real']:.1f}cm",
                (int(obj["p1"][0]), int(obj["p1"][1])-15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

output_img_path = output_dir / f"mediciones_{timestamp}.jpg"
cv2.imwrite(str(output_img_path), img_vis)
print(f"✓ Imagen guardada: {output_img_path}")

# 2. Crear DataFrame con resultados
data = {
    'Tipo': ['Referencia', 'Validación'] + [obj['nombre'] for obj in objetos_medidos],
    'Distancia_px': [dist_ref_px, dist_valid_px] + [obj['dist_px'] for obj in objetos_medidos],
    'Medida_cm': [longitud_real_ref, longitud_medida_valid] + [obj['medida_real'] for obj in objetos_medidos],
    'Incertidumbre_cm': [0, error_absoluto] + [obj['incertidumbre'] for obj in objetos_medidos]
}

df = pd.DataFrame(data)

# Agregar información de calibración
df.attrs['escala'] = escala
df.attrs['error_relativo'] = error_relativo

# Guardar CSV
output_csv_path = output_dir / f"resultados_{timestamp}.csv"
df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
print(f"✓ Resultados guardados: {output_csv_path}")

# Mostrar tabla
print("\nTabla de Resultados:")
print(df.to_string(index=False))

print(f"\n✓ Exportación completada en el directorio: {output_dir}")

## 11. Conclusiones y Recomendaciones

### Conclusiones:

1. **Calibración del Sistema**:
   - La escala se calcula usando un objeto de dimensión conocida
   - Es fundamental que el objeto de referencia sea medido con precisión
   - La calidad de la calibración determina la precisión de todas las mediciones posteriores

2. **Validación e Incertidumbre**:
   - El error relativo calculado representa la incertidumbre del sistema
   - Múltiples validaciones pueden mejorar la estimación de incertidumbre
   - Factores como perspectiva, distorsión de lente y calidad de imagen afectan la precisión

3. **Mediciones**:
   - Las mediciones incluyen la incertidumbre calculada del sistema
   - Para objetos pequeños, el error relativo puede ser más significativo
   - La selección precisa de puntos es crítica para mediciones exactas

### Recomendaciones:

1. **Para mejorar la precisión**:
   - Usar imágenes de alta resolución
   - Tomar fotografías perpendiculares al plano del objeto
   - Minimizar la distorsión de perspectiva
   - Usar múltiples objetos de referencia

2. **Buenas prácticas**:
   - Calibrar con objetos en el mismo plano que los objetos a medir
   - Verificar la calibración con múltiples objetos conocidos
   - Documentar las condiciones de captura de la imagen
   - Realizar múltiples mediciones y promediar resultados

3. **Limitaciones**:
   - Este método asume una proyección ortográfica (sin perspectiva)
   - No corrige distorsión de lente de cámara
   - La precisión disminuye para objetos fuera del plano de referencia
   - Requiere selección manual de puntos

### Aplicaciones:

- Control de calidad en manufactura
- Mediciones en campo para ingeniería civil
- Análisis forense
- Estudios arqueológicos
- Documentación de patrimonio cultural
- Estimación rápida de dimensiones en proyectos
